In [1]:
import numpy as np
from scipy.stats import norm

#### 1
Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины  $ X\sim N\left(\mu,\sigma^2\right) $ лежит в интервале  $ \mu\pm c \cdot \sigma $. Чему равно точное значение константы $ c $? Округлите ответ до четырёх знаков после десятичной точки.

In [2]:
alpha = 1 - 0.997
round(norm.ppf(1 - alpha/2), 4)

2.9677

#### 5
В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо. 

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.


In [3]:
n1 = 11037
n2 = 11034

a1 = 104
a2 = 189

p1 = a1 / n1
p2 = a2 / n2

round(p2 - p1, 4)

0.0077

#### 6
Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [4]:
z = norm.ppf(1 - 0.05 / 2)

left = (p2 - p1) - z * np.sqrt( p1 * (1 - p1) / n1  + p2 * (1 - p2) / n2)
right = (p2 - p1) + z * np.sqrt( p1 * (1 - p1) / n1  + p2 * (1 - p2) / n2)

np.round([left, right], 4)

array([0.0047, 0.0107])

In [5]:
from statsmodels.stats.proportion import confint_proportions_2indep

In [6]:
np.round(confint_proportions_2indep(a2, n2, a1, n1), 4)

array([0.0047, 0.0108])

#### 7
Для бернуллиевских случайных величин $ X \sim Ber(p) $ часто вычисляют величину $ \frac{p}{1-p} $, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $ p $ нужно подставить $ \hat{p} $. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как 
$$ \frac{\frac{189}{11034}}{1 - \frac{189}{11034}} \approx 0.0174 $$
Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [7]:
odds2 = p2 / (1 - p2)
odds1 = p1 / (1 - p1)

In [8]:
round(odds2 / odds1, 4)

1.8321

#### 8
Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:
* составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
* установите random seed=0;
* сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [9]:
aspirin_sample = np.append(np.ones(a1), np.zeros(n1 - a1))
placebo_sample = np.append(np.ones(a2), np.zeros(n2 - a2))

In [10]:
np.random.seed(seed=0)

def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [11]:
def odds(arr):
    p = np.sum(arr) / arr.shape[0]
    return p / (1 - p)

In [12]:
np.random.seed(seed=0)

aspirin_odds_scores = list(map(odds, get_bootstrap_samples(aspirin_sample, 1000)))
placebo_odds_scores = list(map(odds, get_bootstrap_samples(placebo_sample, 1000)))

In [13]:
ratio_odds = list(map(lambda x: x[1] / x[0], zip(aspirin_odds_scores, placebo_odds_scores)))

In [14]:
np.round(stat_intervals(ratio_odds, 0.05), 4)

array([1.4442, 2.3432])